In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn import metrics
import tqdm
import pickle as pkl

In [ ]:
df = pd.read_csv('data/qm8.csv')

In [ ]:
df.head()

In [ ]:
tqdm.tqdm.pandas()

In [ ]:
def process_smile(smile):
    mol = Chem.MolFromSmiles(smile)
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol)
    
    is_3d = False
    pos = []
    for c in mol.GetConformers():
        pos.append(c.GetPositions())
        is_3d |= c.Is3D()
        
    return np.concatenate(pos)

In [ ]:
def smile_features(smile):
    mol = Chem.MolFromSmiles(smile)
    mol = Chem.AddHs(mol)
    
    atomic_nums = [a.GetAtomicNum() for a in mol.GetAtoms()]
    
    dist_matrix = Chem.rdDistGeom.GetMoleculeBoundsMatrix(mol)
    return (atomic_nums, dist_matrix)

In [ ]:
mol = Chem.MolFromSmiles(df.smiles[1])
mol = Chem.AddHs(mol)
AllChem.EmbedMolecule(mol)

In [ ]:
mol

In [ ]:
sample_df = df

In [ ]:
res = df.smiles.head(10).progress_apply(smile_features).to_list()

In [ ]:
sample_df.loc[:, ['Z', 'D']] = sample_df.smiles.progress_apply(smile_features).to_list()

In [ ]:
sample_df.to_json('data/preprocessed.json')

In [ ]:
def gaussian_expansion(D, mu_min=-1, delta_mu=0.2, mu_max=10, sigma=0.2):
    mu = np.arange(mu_min, mu_max + delta_mu, delta_mu)
    diff = D[:,:,np.newaxis] - mu[np.newaxis, np.newaxis, :]
    return np.exp(-diff ** 2 / (2 * sigma))

In [ ]:
sample_df['D_hat'] = sample_df['D'].progress_apply(gaussian_expansion)

In [ ]:
sample_df.to_pickle('data/preprocessed_df.pkl')

In [ ]:
sample_df.to_csv('data/preprocessed.csv', index=False)

In [ ]:
import data
from torch.utils.data import DataLoader

In [ ]:
d = data.QM8Dataset(sample_df.columns[3], 30, 10)

In [ ]:
dl = DataLoader(d, 20)

In [ ]:
Zs, Ds, sizes, target = next(iter(dl))

In [ ]:
Zs.shape, Ds.shape, sizes.shape, target.shape

In [ ]:
D = sample_df.D[0]

In [ ]:
D

In [ ]:
import numpy as np

In [ ]:
gaussian_expansion(D).shape